In [50]:
# score: 0.82666


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [51]:
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import mean_absolute_error, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import optuna

# Get data

In [52]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

X = train_data.copy()
X.drop(['id', 'target'], axis=1, inplace=True)

y = train_data['target']

no_id_test_data = test_data.drop('id', axis=1)

display(X.head())
display(y.head())

,gravity,ph,osmo,cond,urea,calc
0,1.013,6.19,443,14.8,124,1.45
1,1.025,5.40,703,23.6,394,4.18
2,1.009,6.13,371,24.5,159,9.04
3,1.021,4.91,442,20.8,398,6.63
4,1.021,5.53,874,17.8,385,2.21


0    0
1    0
2    0
3    1
4    1
Name: target, dtype: int64

# Feature Engineering

In [53]:
display(X.columns)

def prepXy(df,X_label,y=None):
    
    data = df.drop(['gravity', 'ph', 'cond'], axis=1);
    return data, df, y


print(X.count());

Index(['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc'], dtype='object')

gravity    414
ph         414
osmo       414
cond       414
urea       414
calc       414
dtype: int64


# Preprocess Data

In [54]:
def preprocess_data(X, y=None):
    
    # X = aggregate_features(X);
    
    print(X.columns)
    return prepXy(X, X.columns, y)


'''preprocess data here'''
new_df_train, X, y = preprocess_data(X, y)
new_df_test, no_id_test_data, _ = preprocess_data(no_id_test_data)

display(X.columns)

# cols_with_missing = [col for col in X.columns if X[col].isnull().any()]

# print(f'There are {len(cols_with_missing)} columns with missing values')

# for i in cols_with_missing:
#     print(i)
    
    
# print(X.count());



Index(['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc'], dtype='object')
Index(['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc'], dtype='object')


Index(['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc'], dtype='object')

# Hyperparameter Tuning

In [55]:
def optuna_study(X, y):

    def objective(trial):
        display('hi');
        
        n_estimators = trial.suggest_int('n_estimators', 10, 1000)
        max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 1, 100)
        max_depth = trial.suggest_int('max_depth', 1, 30)
        
        rf = RandomForestClassifier(n_estimators=n_estimators,
                        max_leaf_nodes=max_leaf_nodes,
                        max_depth=max_depth)
        
        # # score = StratifiedKFold(rf, X, y)
        display('hello')
        
        kfold = RepeatedStratifiedKFold(n_splits=40, n_repeats=6, random_state=69);
        
        scores = []
        
        display(kfold.split(X, y))
        
        for train_idx, test_idx in kfold.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_test)
            scores.append(roc_auc_score(y_test, y_pred))
        
        # return np.mean(scores)
        
        display('hehehaha')
        display(scores)
        print(scores)
                        

        # score = cross_val_score(rf, X, y, n_jobs=4, cv=5, scoring='roc_auc')
        return np.mean(scores)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=200)
    
    return study.best_params

# Study 1 (no ph + remove outlier + aggregate values, 100 trials): 0.1564147933997004
# Study 2 (no preprocessing, 100 trials): 0.12094767907088448
# Study 3 (preprocessing kudos to MOHAMMAD RAZEGHI, 200 trials): 0.18433600629551264
# Study 4 (same as 3 but max_depth allowed to be 1, 
# {'n_estimators': 31, 'learning_rate': 0.15241724886836167, 'max_depth': 1}): 0.22565032154969472

# *begin using roc_auc

# Study 5 ({'n_estimators': 109, 'learning_rate': 0.03160535667677366, 'max_depth': 1}): 0.7891310192305856
# Study 6 [I 2023-04-15 17:47:02,473] Trial 199 finished with value: 0.756868939982877 and parameters: {'n_estimators': 70, 'learning_rate': 0.21972623175664083, 'max_depth': 2}. Best is trial 196 with value: 0.7902962715736496.
display('hi2')
best_params = optuna_study(X, y)

display(best_params)

'hi2'

[I 2023-04-17 09:34:21,743] A new study created in memory with name: no-name-44a3440f-b920-4fdd-8b0c-84b7359d072a


'hi'

'hello'

<generator object _RepeatedSplits.split at 0x000001E40F939930>

[W 2023-04-17 09:34:21,763] Trial 0 failed with parameters: {'n_estimators': 487, 'max_leaf_nodes': 32, 'max_depth': 21} because of the following error: KeyError("None of [Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,\n            ...\n            404, 405, 406, 407, 408, 409, 410, 411, 412, 413],\n           dtype='int64', length=403)] are in the [columns]").
Traceback (most recent call last):
  File "c:\Users\andar\VSC Projects\.venv\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\andar\AppData\Local\Temp\ipykernel_17444\1632742153.py", line 24, in objective
    X_train, X_test = X[train_idx], X[test_idx]
                      ~^^^^^^^^^^^
  File "c:\Users\andar\VSC Projects\.venv\Lib\site-packages\pandas\core\frame.py", line 3813, in __getitem__
    indexer = self.columns._get_indexer_strict(key, "columns")[1]
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyError: "None of [Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,\n            ...\n            404, 405, 406, 407, 408, 409, 410, 411, 412, 413],\n           dtype='int64', length=403)] are in the [columns]"

# Submission

In [ ]:
final_model = XGBRegressor(
    n_estimators = best_params['n_estimators'], 
    max_depth = best_params['max_depth'], 
    learning_rate = best_params['learning_rate'],
    n_jobs=4)

final_model.fit(X, y)
predictions = final_model.predict(no_id_test_data)

submission = pd.DataFrame({'id': test_data.id, 'target': predictions})

display(submission.head())

submission.to_csv(f'submission9.csv', index=False)

print('Successful')

,id,target
0,414,0.274909
1,415,0.464385
2,416,0.692975
3,417,0.464385
4,418,0.364252


Successful
